# Cricketer class improvements
A development area where I work out new features to build into the various classes to allow for further analysis. 

In [1]:
# Required imports
import requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

# Import my classes
from classes.Cricketer import Cricketer
from classes.Innings import Innings
from classes.Match import Match

In [2]:
joe_root = 'http://stats.espncricinfo.com/ci/engine/player/303669.html?class=1;template=results;type=allround;view=innings'
root = Cricketer(joe_root)

In [3]:
root_id = '303669'
first_25 = root.innings(True).head(25)

In [4]:
first_innings = first_25.iloc[0].copy()
first_innings_obj = Match(first_innings['match_url']).select_innings(int(first_innings['inns']))

In [5]:
first_innings_scorecard = first_innings_obj.batting()

In [6]:
first_innings_scorecard[first_innings_scorecard.player_id == root_id]

,batsman,how_out,runs,balls_faced,minutes,fours,sixes,strike_rate,is_out,is_keeper,is_captain,player_link,player_id
6,JE Root,c & b Chawla,73,229,289,4,0,31.87,True,False,False,http://www.espncricinfo.com/ci/content/player/...,303669


In [7]:
first_sc_details = first_innings_scorecard[first_innings_scorecard.player_id == root_id]
first_innings['balls_faced'] = int(first_sc_details['balls_faced'].item())
first_innings['position_batted'] = first_sc_details.index.item()
first_innings['minutes'] = first_sc_details.minutes.item()
first_innings['fours'] = first_sc_details.fours.item()
first_innings['sixes'] = first_sc_details.sixes.item()
first_innings['is_captain'] = first_sc_details.is_captain.item()

In [8]:
first_innings

inns                                                               1
score                                                             73
did_bat                                                         True
is_out                                                          True
overs                                                            NaN
conc                                                             NaN
wkts                                                             NaN
did_bowl                                                       False
ct                                                               NaN
st                                                               NaN
opposition                                                     India
ground                                                        Nagpur
start_date                                               13 Dec 2012
match_id                                                      565809
match_url          https://www.esp

With this sort of extra information, we can track other things over time but we will have to start being more aware of the requests that we are doing, as these things take time and resources. 

In [9]:
all_innings = root.innings(True)

In [10]:
print(len(set(all_innings.match_url)))
print(len(all_innings.match_url))

88
331


I think that it makes sense to loop through and create a dictionary of `Match` objects then we aren't going to be creating many unnecessary objects. 

In [11]:
matches = {}
for match_url in set(all_innings.match_url):
    obj = Match(match_url)
    matches[obj.id] = obj

We now have a dictionary of `Match` objects that we can access using the `match_id`. This will allow us to loop through the innings, join to the match and get the extra details for a given innings easily within a loop and doesn't require us to do a request to the page each time. As these matches are historic, we don't need to worry about the stats become out of date in the minutes that we work with this dictionary. 

In [12]:
len(matches)

88

In [13]:
matches

{'565809': <classes.Match.Match at 0x11b9cb4e0>,
 '800463': <classes.Match.Match at 0x11bc4ae80>,
 '800461': <classes.Match.Match at 0x11be6ba90>,
 '667713': <classes.Match.Match at 0x11c2194a8>,
 '1072306': <classes.Match.Match at 0x11c50e1d0>,
 '592397': <classes.Match.Match at 0x11c908b70>,
 '743967': <classes.Match.Match at 0x11cbd9470>,
 '1031437': <classes.Match.Match at 0x11ce97b38>,
 '902637': <classes.Match.Match at 0x11d25c668>,
 '1034809': <classes.Match.Match at 0x11d559780>,
 '1158064': <classes.Match.Match at 0x11d8e3f28>,
 '1119552': <classes.Match.Match at 0x11dc09ba8>,
 '1029827': <classes.Match.Match at 0x11e02d198>,
 '1140386': <classes.Match.Match at 0x11e348a58>,
 '1119553': <classes.Match.Match at 0x11e768e48>,
 '902639': <classes.Match.Match at 0x11eabb240>,
 '566934': <classes.Match.Match at 0x11edd7d30>,
 '1152850': <classes.Match.Match at 0x11f2104e0>,
 '766933': <classes.Match.Match at 0x11f6cfb38>,
 '766931': <classes.Match.Match at 0x11fa86208>,
 '1034815':

In [14]:
matches[first_25.iloc[0].match_id].select_innings(int(first_25.iloc[0].inns)).batting()

,batsman,how_out,runs,balls_faced,minutes,fours,sixes,strike_rate,is_out,is_keeper,is_captain,player_link,player_id
1,AN Cook,lbw b Sharma,1,28,44,0,0,3.57,True,False,True,http://www.espncricinfo.com/ci/content/player/...,11728
2,NRD Compton,c †Dhoni b Sharma,3,12,16,0,0,25.00,True,False,False,http://www.espncricinfo.com/ci/content/player/...,11718
3,IJL Trott,b Jadeja,44,133,168,7,0,33.08,True,False,False,http://www.espncricinfo.com/ci/content/player/...,47623
4,KP Pietersen,c Ojha b Jadeja,73,188,202,10,0,38.82,True,False,False,http://www.espncricinfo.com/ci/content/player/...,19296
5,IR Bell,c Kohli b Chawla,1,28,38,0,0,3.57,True,False,False,http://www.espncricinfo.com/ci/content/player/...,9062
6,JE Root,c & b Chawla,73,229,289,4,0,31.87,True,False,False,http://www.espncricinfo.com/ci/content/player/...,303669
7,MJ Prior,b Ashwin,57,142,179,6,0,40.14,True,True,False,http://www.espncricinfo.com/ci/content/player/...,18675
8,TT Bresnan,lbw b Sharma,0,2,4,0,0,0.00,True,False,False,http://www.espncricinfo.com/ci/content/player/...,9310
9,GP Swann,lbw b Chawla,56,91,96,6,2,61.53,True,False,False,http://www.espncricinfo.com/ci/content/player/...,20431
10,JM Anderson,c Pujara b Chawla,4,17,24,0,0,23.52,True,False,False,http://www.espncricinfo.com/ci/content/player/...,8608
